In [ ]:
from pyincore import IncoreClient, MappingSet, RestorationCurveSet, RestorationService, FragilityService
from pyincore.analyses.waterfacilitydamage import WaterFacilityDamage
from pyincore.analyses.waterfacilityrestoration import WaterFacilityRestoration
import pyincore.globals as pyglobals
from pyincore.models.mapping import Mapping

In [ ]:
# Connect to IN-CORE service
client = IncoreClient()

In [ ]:
# Memphis water facility damage with earthquake
hazard_type = "earthquake"
hazard_id = "5b902cb273c3371e1236b36b"
wf_dataset_id = "5a284f2ac7d30d13bc081e52"
mapping_id = "5b47c383337d4a387669d592"  # PGA and PGD

use_liquefaction = True
use_hazard_uncertainty = False
liq_geology_dataset_id = "5a284f53c7d30d13bc08249c"

# Create and setup water facility damage
wf_dmg = WaterFacilityDamage(client)
wf_dmg.load_remote_input_dataset("water_facilities", wf_dataset_id)

# Load fragility mapping
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
wf_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

result_name = "memphis-wf-dmg"
wf_dmg.set_parameter("result_name", result_name)
wf_dmg.set_parameter("hazard_type", hazard_type)
wf_dmg.set_parameter("hazard_id", hazard_id)
wf_dmg.set_parameter("use_liquefaction", use_liquefaction)
wf_dmg.set_parameter("liquefaction_geology_dataset_id", liq_geology_dataset_id)
wf_dmg.set_parameter("use_hazard_uncertainty", use_hazard_uncertainty)
wf_dmg.set_parameter("num_cpu", 1)

# Run Analysis
wf_dmg.run_analysis()

In [ ]:
# Create and setup water facility restoration
wf_rest = WaterFacilityRestoration(client)

restoration_table_id = "62d05fa61cca614f5243371d"
restoration_mapping_id = "61f075ee903e515036cee0a5"

# Load restoration mapping
restorationsvc = RestorationService(client)
mapping_set = MappingSet(restorationsvc.get_mapping(restoration_mapping_id))  # new format of mapping
wf_rest.load_remote_input_dataset("water_facilities", wf_dataset_id)  # water facility
wf_rest.set_input_dataset('dfr3_mapping_set', mapping_set)
wf_rest.set_input_dataset('damage', wf_dmg.get_output_dataset("result"))
wf_rest.set_parameter("result_name", "memphis_wf_restoration")
wf_rest.set_parameter("discretized_days", [1, 3, 7, 30, 90])
wf_rest.set_parameter("restoration_key", "Restoration ID Code")
wf_rest.set_parameter("end_time", 100.0)
wf_rest.set_parameter("time_interval", 1.0)
wf_rest.set_parameter("pf_interval", 0.05)

In [ ]:
# Run Analysis
wf_rest.run_analysis()

In [ ]:
# Discretized Restoration functionality
func_results = wf_rest.get_output_dataset("func_results").get_dataframe_from_csv()
func_results.head()

In [ ]:
# Functionality change over time
pf_results = wf_rest.get_output_dataset("pf_results").get_dataframe_from_csv()
pf_results.head()

In [ ]:
# Functionality change over time
pf_results = wf_rest.get_output_dataset("pf_results").get_dataframe_from_csv()
pf_results.head()

In [ ]:
# Repair time at certain functionality recovery and damage state
time_results = wf_rest.get_output_dataset("time_results").get_dataframe_from_csv()
time_results.head()

### Example of how to use the utility function to calculate repair time and functionality

In [ ]:
from pyincore.analyses.waterfacilityrestoration import WaterFacilityRestorationUtil

In [ ]:
# Functionality for particular component and damage state
wf_util = WaterFacilityRestorationUtil(wf_rest)
functionality = wf_util.get_percentage_func(guid="e1bce78d-00a1-4605-95f3-3776ff907f73",
                                            damage_state="DS_0", time=2.0)
functionality

In [ ]:
# Repair time for particular component and damage state
time = wf_util.get_restoration_time(guid="e1bce78d-00a1-4605-95f3-3776ff907f73", damage_state="DS_1", pf=0.81)
time